In [48]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

In [57]:
pd.set_option('display.max_columns', None)

df_origin = pd.read_csv('Android_Malware_clean_eda.csv', low_memory=False)
df = df_origin.copy()

In [58]:
df['Timestamp'] = pd.to_datetime(df['Timestamp']).astype('int64') // 10**9

In [59]:
categorical_columns = ['Source IP', 'Destination IP', 'Label']

# Enumerate and create new columns
for column in categorical_columns:
    df[column + '_ID'] = df[column].astype('category').cat.codes

df_num = df.drop(columns=categorical_columns)
df_num = df_num.drop(columns='Flow ID')
df_num

,Source Port,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Source IP_ID,Destination IP_ID,Label_ID
0,50004,443.0,6.0,1497354759,37027,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,54.014638,3.702700e+04,0.000000e+00,37027.0,37027.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,32,32,27.007319,27.007319,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,1.0,0.0,1.0,0.0,2994.0,362.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9,1276,0
1,35455,443.0,6.0,1497354759,36653,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,54.565793,3.665300e+04,0.000000e+00,36653.0,36653.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,32,32,27.282896,27.282896,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,1.0,0.0,1.0,0.0,2860.0,362.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9,1276,0
2,51775,443.0,6.0,1497354762,534099,8,12,1011.0,11924.0,581.0,0.0,126.375000,207.799311,1460.0,0.0,993.666667,656.474376,24218.356522,37.446241,2.811047e+04,4.314810e+04,158676.0,8.0,481340.0,6.876286e+04,6.237618e+04,158676.0,449.0,487990.0,44362.727273,86342.042540,217790.0,8.0,0.0,180,244,14.978496,22.467745,0.0,1460.0,615.952381,672.744786,452585.547619,0.0,0.0,1.0,0.0,0.0,1.0,646.750000,126.375000,993.666667,8.0,1011.0,12.0,11924.0,65535.0,63441.0,3.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9,642,0
3,51775,443.0,6.0,1497354763,9309,3,0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,322.268772,4.654500e+03,5.137131e+03,8287.0,1022.0,9309.0,4.654500e+03,5.137131e+03,8287.0,1022.0,0.0,0.000000,0.000000,0.0,0.0,0.0,60,0,322.268772,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.000000,0.000000,3.0,0.0,0.0,0.0,65535.0,-1.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9,642,0
4,51776,443.0,6.0,1497354762,19890496,8,6,430.0,5679.0,218.0,0.0,53.750000,99.538578,1460.0,0.0,946.500000,710.412204,307.131607,0.703854,1.530038e+06,5.377887e+06,19426714.0,7.0,19890496.0,2.841499e+06,7.314093e+06,19426714.0,639.0,410964.0,82192.800000,154845.683018,355945.0,7.0,0.0,180,124,0.402202,0.301652,0.0,1460.0,407.266667,626.958471,393076.923810,0.0,0.0,1.0,0.0,0.0,0.0,436.357143,53.750000,946.500000,8.0,430.0,6.0,5679.0,65535.0,64022.0,2.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9,642,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353408,80,38405.0,6.0,1497662951,126711,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,15.783949,1.267110e+05,0.000000e+00,126711.0,126711.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,32,32,7.891975,7.891975,0.0,0.0,0.00000

In [39]:
X = df_num.drop('Label_ID', axis=1)
y = df_num['Label_ID']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [44]:
lasso_model = Lasso(alpha=0.3, max_iter=10000)

In [45]:
lasso_model.fit(X_train, y_train)

C:\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.214e+05, tolerance: 2.852e+01
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=0.3, max_iter=10000)

In [47]:
selected_features = X.columns[lasso_model.coef_ != 0]
coefficients = lasso_model.coef_[lasso_model.coef_ != 0]

print("Selected Features:")
for feature, coef in zip(selected_features, coefficients):
    print(f"{feature}: {coef}")

print("\nFeatures Not Selected:")
not_selected_features = X.columns[lasso_model.coef_ == 0]
for feature in not_selected_features:
    print(feature)

Selected Features:
Source Port: 9.212264200455038e-07
Destination Port: 8.672409790694748e-07
Timestamp: 3.7244599668779833e-07
Flow Duration: 2.2375503267252064e-09
Total Length of Fwd Packets: 8.312434974101361e-08
Total Length of Bwd Packets: -2.793764675260279e-09
Fwd Packet Length Max: 3.121916982685734e-05
Fwd Packet Length Min: -0.00012313514196204546
Fwd Packet Length Mean: -5.399521842756286e-05
Fwd Packet Length Std: -0.0002064600725760509
Bwd Packet Length Max: 1.7697127102782092e-05
Bwd Packet Length Std: -0.00013351181167220805
Flow Bytes/s: -9.363285546231036e-09
Flow IAT Mean: -1.1595292023330422e-08
Flow IAT Std: 2.2253246155734627e-08
Flow IAT Max: -7.955169333343115e-09
Flow IAT Min: 2.0659375913891584e-08
Fwd IAT Total: -4.009434717099391e-09
Fwd IAT Mean: 1.9031750705189496e-08
Fwd IAT Std: -1.5229437557567465e-08
Fwd IAT Max: 5.761201572386723e-09
Fwd IAT Min: -2.5447444485908572e-08
Bwd IAT Total: 2.4362384042785166e-09
Bwd IAT Mean: -7.450455779450794e-09
Bwd IAT

In [52]:
predictions = lasso_model.predict(X_test)
binary_predictions = (predictions > 0.5).astype(int)

accuracy = accuracy_score(y_test, binary_predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.18956467608901717


## with columns removed

In [54]:
df_num.columns

Index(['Source Port', 'Destination Port', 'Protocol', 'Timestamp',
       'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets',
       'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s',
       'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length',
       'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance',
       'FIN Flag Count', 'SYN Flag Count', 'PSH Flag Co

In [60]:
columns_to_drop = [
    'Flow Duration', 'Fwd Packet Length Mean', 'Bwd Packet Length Mean',
    'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std',
    'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Mean', 'Fwd IAT Std',
    'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Mean', 'Bwd IAT Std',
    'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd Header Length',
    'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
    'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
    'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
    'SYN Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count',
    'Down/Up Ratio', 'Average Packet Size', 'Avg Fwd Segment Size',
    'Avg Bwd Segment Size', 'Subflow Fwd Packets', 'Subflow Fwd Bytes',
    'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'Init_Win_bytes_forward',
    'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'min_seg_size_forward',
    'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean',
    'Idle Max', 'Idle Min', 'Source IP_ID', 'Destination IP_ID'
]

# Drop the specified columns
df_num = df_num.drop(columns=columns_to_drop)

In [61]:
X = df_num.drop('Label_ID', axis=1)
y = df_num['Label_ID']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [62]:
lasso_model.fit(X_train, y_train)

Lasso(alpha=0.3, max_iter=10000)

In [63]:
selected_features = X.columns[lasso_model.coef_ != 0]
coefficients = lasso_model.coef_[lasso_model.coef_ != 0]

print("Selected Features:")
for feature, coef in zip(selected_features, coefficients):
    print(f"{feature}: {coef}")

print("\nFeatures Not Selected:")
not_selected_features = X.columns[lasso_model.coef_ == 0]
for feature in not_selected_features:
    print(feature)

Selected Features:
Source Port: 7.78278144384105e-07
Destination Port: 1.1402682324808312e-06
Timestamp: 3.734374100632748e-07
Total Length of Fwd Packets: 1.2302591682126892e-07
Total Length of Bwd Packets: -1.5550370903646627e-08
Fwd Packet Length Max: 3.268882012052424e-05
Fwd Packet Length Min: -3.4978764718589054e-05
Fwd Packet Length Std: -0.00014757209628384426
Bwd Packet Length Max: 9.398471973288274e-05
Bwd Packet Length Std: -9.977945110465696e-05
Fwd IAT Total: -4.099926179986556e-10
Bwd IAT Total: 1.356480518830915e-09
Idle Std: -1.5723469140998557e-09

Features Not Selected:
Protocol
Total Fwd Packets
Total Backward Packets
Bwd Packet Length Min


In [64]:
predictions = lasso_model.predict(X_test)
binary_predictions = (predictions > 0.5).astype(int)

accuracy = accuracy_score(y_test, binary_predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.18868752033728053
